In [46]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [47]:
compute_device = torch.device('cuda')

In [48]:
class FullyConnectedRNN(nn.Module):
    def __init__(self, hidden_units):
        super(FullyConnectedRNN, self).__init__()
        
        self.parameter_list = nn.ParameterList()
        
        self.hidden_units = hidden_units
        
        self.hidden_state = torch.zeros(hidden_units).to(compute_device)
        self.W = nn.Parameter(torch.randn(self.hidden_units, self.hidden_units)).to(compute_device)
        self.relu = nn.ReLU()
        
        self.parameter_list.append(self.W)
        
        # Define the RNN layer
        # self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        
        # Define the fully connected layer
        # self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        
        # solution 1: architectural - input-segregated populatons
        input_matrix = torch.ones(self.hidden_units).to(compute_device)
        input_matrix[:self.hidden_units//2] *= x[0]
        input_matrix[self.hidden_units//2:] *= x[1]
        
        preactivations = self.hidden_state @ self.W + input_matrix
        self.hidden_state = self.relu(preactivations)
        
        # Initialize hidden state with zeros
        # h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()
        
        # Forward propagate RNN
#         out, _ = self.rnn(x, h0.detach())
        
#         # Decode the hidden state at the last time step
#         out = self.fc(out[:, -1, :])
        
        return preactivations, self.hidden_state.detach(), self.W

# Example usage
# input_size = 10  # Number of features in the input
hidden_size = 100  # Number of hidden units in the RNN layer
# num_layers = 2  # Number of RNN layers
# output_size = 1  # Number of output units

# Initialize the model

# # Define a sample input tensor (batch_size, sequence_length, input_size)
# sample_input = torch.randn(1, 5, input_size)

# # Forward pass
# output = model(sample_input)
# print(output)


In [49]:
# FUNCTIONS
def generate_acceleration_and_velocity(samples, initial_velocity = 1):
    
    acceleration = np.around(np.random.normal(0,1, samples), 2)
    
    velocity = [initial_velocity]
    const = 0.1
    for i in range(acceleration.shape[0]):
        velocity.append(velocity[-1] + const*acceleration[i])
    
    return acceleration, velocity

def generate_variable_sine(samples, velocity):
    def get_freq_per_sample(rad_angle, dt):
        return 2 * np.pi * rad_angle * dt

    t = np.linspace(0, 1, samples)
    dt = t[1]-t[0]
    # print(dt)
    y = list()
    phi = 0
    for i in range(t.shape[0]):
        c = np.cos(phi)
        y.append(c)
        phi = phi + get_freq_per_sample(10*velocity[i], dt)
    return t, y

In [50]:
def generate_inputs(samples):
    acceleration, velocity = generate_acceleration_and_velocity(samples)
    t, sine_signal = generate_variable_sine(samples, velocity)
    return acceleration, sine_signal

In [51]:
SAMPLE_NUM = 1000
acceleration, velocity = generate_acceleration_and_velocity(SAMPLE_NUM)
t, sine_signal = generate_variable_sine(SAMPLE_NUM, velocity)

### Training

In [52]:
def L1_loss_function(x):
    return torch.mean(torch.abs(x))

In [53]:
model = FullyConnectedRNN(100)

criterion = nn.MSELoss()

learning_rate = 0.00001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [54]:
losses = []
train_outs = {}

samples = 1000

for epoch in range(500):
    loss = torch.zeros(1, dtype=torch.float, requires_grad=True).to(compute_device)
    optimizer.zero_grad()
    
    train_outs[f'epoch{epoch}'] = {}
    
    acceleration, sine_signal = generate_inputs(samples)
    for t in range(len(acceleration)):
        preactivations, hidden_state, W = model([acceleration[t], sine_signal[t]])
        # train_outs[f'epoch{epoch}']['preactivations'] = preactivations
        # train_outs[f'epoch{epoch}']['hidden_state'] = hidden_state
        # train_outs[f'epoch{epoch}']['W'] = W
        loss = loss + L1_loss_function(preactivations) 

    # loss = criterion(output_HD_sequence, target_HD_sequence)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())
    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

Epoch 1, Loss: nan


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.